In [2]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
from pylab import plot, show, text
import datetime
import matplotlib.dates as mdates
import pylab
%matplotlib inline


In [3]:
with open('amazon-meta.txt', 'r', encoding="utf8") as f:
    file_content = f.read()
f.close()

In [4]:
sample = []
for string in file_content.split('\n')[:30]:
    sample.append(string)
    
sample
# The first 3 line is descriptive
# Each product is separated with '',


['# Full information about Amazon Share the Love products',
 'Total items: 548552',
 '',
 'Id:   0',
 'ASIN: 0771044445',
 '  discontinued product',
 '',
 'Id:   1',
 'ASIN: 0827229534',
 '  title: Patterns of Preaching: A Sermon Sampler',
 '  group: Book',
 '  salesrank: 396585',
 '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
 '  categories: 2',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
 '  reviews: total: 2  downloaded: 2  avg rating: 5',
 '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
 '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5',
 '',
 'Id:   2',
 'ASIN: 0738700797',
 '  title: Candlemas: Feast of Flames',
 '  group: Book',
 '  salesrank: 168596',
 '  similar: 5  0738700827  1567184960  1567182836  0738

In [5]:
#The following two lists are created to save data of each product. 
#product list contains each info of a product, then this set will append to df.
data = []
product = []

for string in file_content.split('\n')[3:]:
    if string != '': #till the '' every info belongs to the same product
        product.append(string)
    else: 
        data.append(product)
        product = []

In [6]:
data[0:5]
#the data is more readeble now, still needs to extract certain info for each product. 

[['Id:   0', 'ASIN: 0771044445', '  discontinued product'],
 ['Id:   1',
  'ASIN: 0827229534',
  '  title: Patterns of Preaching: A Sermon Sampler',
  '  group: Book',
  '  salesrank: 396585',
  '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
  '  categories: 2',
  '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
  '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
  '  reviews: total: 2  downloaded: 2  avg rating: 5',
  '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
  '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5'],
 ['Id:   2',
  'ASIN: 0738700797',
  '  title: Candlemas: Feast of Flames',
  '  group: Book',
  '  salesrank: 168596',
  '  similar: 5  0738700827  1567184960  1567182836  0738700525  0738700940',
  '  categories: 2',
  '   |Books[283155]|Subjects[1000]|Re

In [ ]:
#playing with data
P1 = ['Id:   1',
  'ASIN: 0827229534',
  '  title: Patterns of Preaching: A Sermon Sampler',
  '  group: Book',
  '  salesrank: 396585',
  '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
  '  categories: 2',
  '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
  '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
  '  reviews: total: 2  downloaded: 2  avg rating: 5',
  '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
  '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5']

P1[9]

The data set named 'data' is a list of items where each item represents each product's information. 

# 1. Trustworthiness of ratings
Ratings are susceptible to manipulation, bias etc. What can you say (quantitatively speaking) about the ratings in this dataset?


In [7]:
# From set df, ratings need to be extracted. 
df_ratings = {}

for product in data:
    Id = ''
    group = ''
    total_reviews = ''
    downloaded_review =''
    avg_rating = ''
    
    for info in product:
        if info.startswith('Id:'):
            Id = info.split()[-1]
        elif info.startswith('  group:'):
            group = info.split()[-1]
        elif info.startswith('  reviews:'):
            total_reviews = info.split()[2]
            downloaded_review = info.split()[4]
            avg_rating = info.split()[7]
        elif info.startswith('  discontinued product'):
            skip = True
    if skip == False:
        df_ratings[Id]= [group, total_reviews, downloaded_review, avg_rating]
    else:
        skip = False
    

In [8]:
    
ratings = pd.DataFrame.from_dict(df_ratings)
ratings = ratings.T.reset_index(drop=False)
ratings.columns = ['id','group', 'total', 'downloaded', 'avg_rating']
ratings = ratings.iloc[1:]
ratings['id']  = ratings['id'].astype(int)
ratings['total']  = ratings['total'].astype(int)
ratings['downloaded']  = ratings['downloaded'].astype(int)
ratings['avg_rating']  = ratings['avg_rating'].astype(float)
ratings = ratings.sort_values('id')
ratings.head()

,id,group,total,downloaded,avg_rating
1,1,Book,2,2,5.0
109902,2,Book,12,12,4.5
219674,3,Book,1,1,5.0
329477,4,Book,1,1,4.0
439371,5,Book,0,0,0.0


In [9]:
ratings.describe()

,id,total,downloaded,avg_rating
count,542684.000000,542684.000000,542684.000000,542684.000000
mean,274414.212208,14.339818,13.992017,3.209534
std,158454.479276,74.768369,71.908413,1.996296
min,1.000000,0.000000,0.000000,0.000000
25%,137161.750000,0.000000,0.000000,0.000000
50%,274427.500000,2.000000,2.000000,4.000000
75%,411674.250000,8.000000,7.000000,5.000000
max,548551.000000,5545.000000,4995.000000,5.000000


# Total reviews with respect to categories
Books are the most reviewed items followed by music, DVD and videos. The percentage of products without review is 25%.

When average ratings of each categories are investigated, it is seen that product, software and sports categories have larger ratings though the number of reviews for these categories are small. This shows that consumers who reviewed these products are the ones who have positive experiences with the product, i.e. mostly happy customers make reviews for these products. 

The lowest average rating categories are consumer electronics and games. These two categories also have the minimum number of reviews which shows that unhappy or unsatisfied customers mostly make reviews at these categories. 

Lastly, the categories with larger number of reviews have average ratings around 3 -3.5. 

In [12]:
#products without review
print(len(ratings[ratings['total']==0])/len(ratings))

0.2578830405908411


In [13]:
ratings.groupby(['group'])['total'].sum()

group
Book        4707583
CE                5
DVD          834312
Games             3
Music       1411619
Product           3
Software         13
Sports            5
Toy              37
Video        828410
Name: total, dtype: int32

In [14]:
ratings.groupby(['group'])['avg_rating'].mean()

group
Book        3.051148
CE          1.750000
DVD         3.670996
Games       2.500000
Music       3.635529
Product     4.500000
Software    4.500000
Sports      4.000000
Toy         3.812500
Video       3.563105
Name: avg_rating, dtype: float64

# 2. Category bloat 
Consider the product group named 'Books'. Each product in this group is associated with categories. Naturally, with categorization, there are tradeoffs between how broad or specific the categories must be.
For this dataset, quantify the following:

a. Is there redundancy in the categorization? How can it be identified/removed?

**Looking at the data closer, it can easily be seen that there is redundancy in the categorization. For example, product with Id:1, i.e. data[1], has 2 subcategories. Similarly, product with Id:4, i.e. data[4], has 5 subcategories where each is very similar to another. **


b. Is it possible to reduce the number of categories drastically (say to 10% of existing categories) by sacrificing relatively few category entries (say close to 10%)?


**There are 12,853 number of unique categories for 393,561 books whereas the number of categories is 1,440,329. Each book has on average 3.7 categories. Reducing the number of categories will probably be useful, will not hurt searching. 'categories' data also shows that some sub_category names are authors name. **


In [15]:
data[0:10]

[['Id:   0', 'ASIN: 0771044445', '  discontinued product'],
 ['Id:   1',
  'ASIN: 0827229534',
  '  title: Patterns of Preaching: A Sermon Sampler',
  '  group: Book',
  '  salesrank: 396585',
  '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
  '  categories: 2',
  '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
  '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
  '  reviews: total: 2  downloaded: 2  avg rating: 5',
  '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
  '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5'],
 ['Id:   2',
  'ASIN: 0738700797',
  '  title: Candlemas: Feast of Flames',
  '  group: Book',
  '  salesrank: 168596',
  '  similar: 5  0738700827  1567184960  1567182836  0738700525  0738700940',
  '  categories: 2',
  '   |Books[283155]|Subjects[1000]|Re

In [16]:
data[1]


['Id:   1',
 'ASIN: 0827229534',
 '  title: Patterns of Preaching: A Sermon Sampler',
 '  group: Book',
 '  salesrank: 396585',
 '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
 '  categories: 2',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
 '  reviews: total: 2  downloaded: 2  avg rating: 5',
 '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
 '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5']

In [17]:
data[4]

['Id:   4',
 'ASIN: 0842328327',
 '  title: Life Application Bible Commentary: 1 and 2 Timothy and Titus',
 '  group: Book',
 '  salesrank: 631289',
 '  similar: 5  0842328130  0830818138  0842330313  0842328610  0842328572',
 '  categories: 5',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Reference[172810]|Commentaries[12155]|New Testament[12159]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Christian Living[12333]|Discipleship[12335]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Bibles[12059]|Translations[764432]|Life Application[572080]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Bible & Other Sacred Texts[12056]|Bible[764430]|New Testament[572082]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Bibles[12059]|Study Guides, History & Reference[764438]|General[572094]',
 '  reviews: total: 1  downloaded: 1  avg ra

In [18]:
categories = []
total_book = 0 

for product in data:
    group = ''
    for item in product:
        if item.startswith('  group:'):
            group = item.split()[-1]
        if group == 'Book':
            if item.startswith('   |'):
                categories.append(item.strip())
    if group == 'Book':
        total_book += 1 


In [20]:
#total number of categories
print(len(categories))


1440329


In [21]:
#total number of unique categories
print(len(set(categories)))

12853


In [22]:
total_book

393561

In [23]:
sub_categories = []
for item in categories:
    for sub in item.split('|')[1:]:
        sub_categories.append(sub)

In [24]:
from collections import Counter
categories = Counter(sub_categories)


In [25]:
categories = pd.DataFrame.from_dict(categories, orient='index').reset_index()

In [26]:
categories.sort_values(0)

,index,0
14922,Unlikely Heroes[14155061],1
12423,All Works by Elgar[18108],1
14041,Basketball[2519],1
12424,( F )[18353],1
9906,Cyprus[4944],1
14042,Basketball[733936],1
9909,The Cooper Kids Adventures[280259],1
12425,"Franck, César[19108]",1
12422,"Elgar, Sir Edward[18091]",1
12426,General[19118],1


# Part B
Give the number crunching a rest! Just think about these problems.

1. Algorithm thinking: 

How would you build the product categorization from scratch, using similar/co-purchased information?

 **To create a new product categorization from scratch, Natural Language Toolkit and a clustering algorithm like Support Vector Machine can be used to identify most common categories. **
    
2. Product thinking:

Now, put on your 'product thinking' hat:

a. Is it a good idea to show users the categorization hierarchy for items?

**It may be a good idea to show users the categorization hierarchy for items. However, it should be noted that the number of categories should limited otherwise it might be overwhelmed to users. **
   
b. Is it a good idea to show users similar/co-purchased items?    
c. Is it a good idea to show users reviews and ratings for items?

**It is always good idea to show users similar/co-purchased items since it may help a user while making shopping decision or widen his/her shopping perspective/experience. Similarly, showing users reviews and ratings for the items might also be helpful a user's shopping decision. **
    
